## Long Form Summarization using `text-bison 8k`, `32k` and `DocAI`

ℹ️ **Note:** This leverages the table of contents (TOC) from the documents.

<span style="color:blue; font-weight:bold">Setup</span>
- _Import Libraries_: Get all the necessary Python libraries onboard.
- _Initialize Logging_: Make sure we have a log of all the operations.

<span style="color:green; font-weight:bold">Configuration</span>
- _Define Parameters_: Lay out all the essential parameters for our operations.
- _Authentication_: Authenticate the user and ensure security.
- _Setup Clients_: Initialize the API clients for our services.

<span style="color:purple; font-weight:bold">Document Preprocessing</span>
- _Split PDF_: Break the PDF into manageable chunks for better OCR results.

<span style="color:orange; font-weight:bold">Document OCR</span>
- _Convert to Text_: Use DocAI to transcribe the PDF content into text format.

<span style="color:red; font-weight:bold">Summarization</span>
- _Map Reduce 1 uses `text-bison-8k`_
- _Map Reduce 2 and 3 use `text-bison-32k`_

<span style="color:teal; font-weight:bold">Final Consolidation</span>
- _Refine Summary_: Make sure the summary is crisp and to the point.
- _Save_: Store the final summary for future reference.

<span style="color:brown; font-weight:bold">Focused Summarization</span>
- _Extract Sections_: Dive deeper and pull out specific sections from the final summary for a more focused understanding.


#### Imports 

In [1]:
from vertexai.preview.language_models import TextGenerationModel
from google.api_core.client_options import ClientOptions
from concurrent.futures import ThreadPoolExecutor
from google.cloud import documentai
from pypdf import PdfWriter
from pypdf import PdfReader 
from tqdm import tqdm
import tiktoken
import vertexai
import requests
import logging
import json
import os

##### Setup logging 

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

#### Essentials 

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './../credentials/vai-key.json'
access_token = !gcloud auth print-access-token

In [4]:
PROJECT_ID = 'arun-genai-bb'
LOCATION = 'us-central1'
MODEL_NAME_8K = 'text-bison@latest'
MODEL_NAME_32K = 'text-bison-32k@latest'
# May need to request quota upgrade https://console.cloud.google.com/iam-admin/quotas?project=vertex-pe-only Service: Vertex AI API, text-bison-32k region:us-central1
ENCODING_NAME = 'cl100k_base'
CONTEXT_LENGTH = 32000  # text-bison-32k
STREAMING_API_URL_8K = f'https://us-central1-aiplatform.googleapis.com/ui/projects/{PROJECT_ID}/locations/us-central1/publishers/google/models/{MODEL_NAME_8K}:serverStreamingPredict'
STREAMING_API_URL_32K = f'https://us-central1-aiplatform.googleapis.com/ui/projects/{PROJECT_ID}/locations/us-central1/publishers/google/models/{MODEL_NAME_32K}:serverStreamingPredict'
DOCAI_PROCESSOR_NAME = 'projects/390991481152/locations/us/processors/ad9557a5be49204e'  # copy from notebook 00
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [5]:
client_options = ClientOptions(api_endpoint=f'us-documentai.googleapis.com')
docai_client = documentai.DocumentProcessorServiceClient(client_options=client_options)

In [6]:
encoder = tiktoken.get_encoding(ENCODING_NAME)
logger.info(f'Using encoder=={encoder.name}')

Using encoder==cl100k_base


#### Use Google DocumentAI to process input PDF

##### Break PDF into smaller PDFs for OCR

In [7]:
LOCAL_INPUT_DIR = './DATA/INPUT'
LOCAL_OUTPUT_DIR = './DATA/OUTPUT'
FILE_NAME = 'file-2'

In [8]:
reader = PdfReader(f'{LOCAL_INPUT_DIR}/{FILE_NAME}.pdf')
pages = {}

for i, page in enumerate(reader.pages):
    pages[i] = page

In [9]:
n = len(reader.pages)
d = 15  # docai has a current constraint of 15 pages per document 

for i in range(0, n, d):
    writer = PdfWriter()
    for j in range(i, i+d):
        if j < n:
            writer.add_page(pages[j])
    os.makedirs(f'{LOCAL_INPUT_DIR}/{FILE_NAME}/PARTS/', exist_ok=True)
    with open(f'{LOCAL_INPUT_DIR}/{FILE_NAME}/PARTS/{FILE_NAME}_{i+1}-{i+d}.pdf', 'wb') as f:
        writer.write(f)

In [10]:
def layout_to_text(layout: documentai.Document.Page.Layout, text: str) -> str:
    """
    Document AI identifies text in different parts of the document by their
    offsets in the entirety of the document's text. This function converts
    offsets to a string.
    """
    # If a text segment spans several lines, it will be stored in different text segments.
    return ''.join(text[int(segment.start_index): int(segment.end_index)] for segment in layout.text_anchor.text_segments)

In [11]:
def get_file_paths(dir_name: str) -> list:
    file_paths = []
    for file_name in os.listdir(dir_name):
        if os.path.isfile(os.path.join(dir_name, file_name)):
            file_path = os.path.join(dir_name, file_name)
            file_paths.append(file_path)
    return file_paths

In [12]:
def ocr_docai(file_path: str) -> dict:
    pages_map = {}

    with open(file_path, 'rb') as f:
        pdf = f.read()
        raw_document = documentai.RawDocument(content=pdf, mime_type='application/pdf')
        request = documentai.ProcessRequest(name=DOCAI_PROCESSOR_NAME, raw_document=raw_document)
        response = docai_client.process_document(request=request)
        text = response.document.text
        file_name = file_path.split('/')[-1]
        page_number = int(file_name.split('.')[0].split('-')[-1])
        for page in response.document.pages:
            page_text = []
            for paragraph in page.paragraphs:
                paragraph_text = layout_to_text(paragraph.layout, text)
                page_text.append(paragraph_text)
            pages_map[page_number] = ''.join(page_text)
            page_number += 1
    return pages_map

In [13]:
%%time 

input_dir = f'./DATA/INPUT/{FILE_NAME}/PARTS/'
file_paths = get_file_paths(input_dir)
    
pages_map_list = []
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:  
    pages_map_list = list(tqdm(executor.map(ocr_docai, file_paths)))

merged_dict = {k: v for d in pages_map_list for k, v in d.items()}   
sorted_pages_map = dict(sorted(merged_dict.items()))

pages = []
for _, page_text in sorted_pages_map.items():
    pages.append(page_text)

73it [00:48,  1.49it/s]

CPU times: user 3.8 s, sys: 3.13 s, total: 6.93 s
Wall time: 49 s


Save concatenated pages as txt for later use (if needed)

In [14]:
extracted_pages = ''.join(pages)
os.makedirs(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/VAI/', exist_ok=True)
with open(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/VAI/{FILE_NAME}.txt', 'w') as out:
    out.write(extracted_pages)

In [15]:
def get_total_tokens(contexts: list) -> int:
    total_tokens = 0
    for context in contexts:
        n_tokens = len(encoder.encode(context))
        total_tokens += n_tokens 
    return total_tokens

In [16]:
total_tokens = get_total_tokens([extracted_pages])
logger.info(f'Total tokens in the input doc = {total_tokens}')

Total tokens in the input doc = 414765


In [17]:
def get_max_tokens_per_page(contexts: list) -> list:
    max_tokens_per_page = 0
    for context in contexts:
        n_tokens = len(encoder.encode(context))
        if n_tokens > max_tokens_per_page:
            max_tokens_per_page = n_tokens
    return max_tokens_per_page

#### Map Reduce 1

In [18]:
def get_summary_via_streaming_api_mr1(chunk: str) -> str:
    prompt = f'You are a Financial Regulations & Derivatives Expert. Summarize the following information a minimum set of unordered key bullet points in English, capturing the essential details.\n\n{chunk}'
    headers = {
        "Authorization": f"Bearer {access_token[0]}",
        "Content-Type": "application/json; charset=utf-8"
    }
    
    data = {
        "inputs": [
            {
                "struct_val": {
                    "prompt": {
                        "string_val": [prompt]
                    }
                }
            }
        ],
        "parameters": {
            "struct_val": {
                "temperature": {"float_val": 0.0},
                "maxOutputTokens": {"int_val": 256},
                "topK": {"int_val": 40},
                "topP": {"float_val": 0.8}
            }
        }
    }
    response = requests.post(STREAMING_API_URL_8K, headers=headers, json=data)
    content = json.loads(response.content)
    output = []

    for item in content:
        try:
            text = item['outputs'][0]['structVal']['content']['stringVal'][0]
            output.append(text)
        except Exception as e:
            logger.error(f'Content error => {content}')
    output = ''.join(output)
    return output


In [19]:
CONTEXTS_PER_CALL = 5  # process 5 pages per API call
MAX_OUTPUT_TOKENS = 256

def reduce_mr1(contexts: list) -> list:
    partitions = []
    max_input_tokens = CONTEXT_LENGTH - MAX_OUTPUT_TOKENS
    logger.info(f'Max input tokens allowed per API call = {max_input_tokens}')
    max_tokens_per_page = get_max_tokens_per_page(contexts)
    logger.info(f'Max tokens per page = {max_tokens_per_page}')
    logger.info(f'Processing {CONTEXTS_PER_CALL} pages per API call')
    
    for i in range(0, len(contexts), CONTEXTS_PER_CALL):
        partitions.append(contexts[i: i+CONTEXTS_PER_CALL])

    chunks = []
    for partition in partitions:
        chunks.append('\n'.join(partition))

    reduced_contexts = []

    # max_workers can result in running over quota limits for invocation | current limit for text bison is 60/min
    # for our experiments, we set max_workers=4 cores without any limit breach
    with ThreadPoolExecutor(max_workers=4) as executor:  
        reduced_contexts = list(tqdm(executor.map(get_summary_via_streaming_api_mr1, chunks),  total=len(chunks)))
    return reduced_contexts



In [20]:
logger.info(f'Number of pages to process = {len(pages)}')
summaries = reduce_mr1(pages)
logger.info(f'Number of generated summaries = {len(summaries)}')
n_tokens = get_total_tokens(summaries)
logger.info(f'Total number of tokens in generated summaries = {n_tokens}')

Number of pages to process = 1089
Max input tokens allowed per API call = 31744
Max tokens per page = 722
Processing 5 pages per API call
 20%|██        | 44/218 [01:03<04:26,  1.53s/it]

In [ ]:
logger.info(summaries[5])
logger.info('-' * 100)
logger.info(summaries[15])
logger.info('-' * 100)
logger.info(summaries[20])

##### Persist internediate summaries (Map Reduce 1) to local disk

In [ ]:
logger.info(f'Total number of summaries = {len(summaries)}')

In [ ]:
for i, summary in enumerate(summaries):
    os.makedirs(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/VAI/MAP_REDUCE_1/', exist_ok=True)
    with open(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/VAI/MAP_REDUCE_1/summary-{i}.txt', 'w') as f:
        f.write(summary)

#### Map Reduce 2

Injecting **table of content** into the prompt

In [ ]:
toc = "I. Introduction\nA. Overview of the Proposal\nB. Use of Internal Models Under the\nProposed Framework\nII. Scope of Application\nIII. Proposed Changes to the Capital Rule\nA. Calculation of Capital Ratios and\nApplication of Buffer Requirements\n1. Standardized Output Floor\n2. Stress Capital Buffer Requirement\nB. Definition of Capital\n1. Accumulated Other Comprehensive\nIncome\n2. Regulatory Capital Deductions\n3. Additional Definition of Capital\nAdjustments\n4. Changes to the Definition of Tier 2\nCapital Applicable to Large Banking\nOrganizations\nC. Credit Risk\n1. Due Diligence\n2. Proposed Risk Weights for Credit Risk\n3. Off-Balance Sheet Exposures\n4. Derivatives\n5. Credit Risk Mitigation\nD. Securitization Framework\n1. Operational Requirements\n2. Securitization Standardized Approach\n(SEC–SA)\n3. Exceptions to the SEC–SA Risk-Based\nCapital Treatment for Securitization\nExposures\n4. Credit Risk Mitigation for Securitization\nExposures\nE. Equity Exposures\n1. Risk-Weighted Asset Amount\nF. Operational Risk\n1. Business Indicator\n2. Business Indicator Component\n3. Internal Loss Multiplier\n4. Operational Risk Management and Data\nCollection Requirements\nG. Disclosure Requirements\n1. Proposed Disclosure Requirements\n2. Specific Public Disclosure Requirements\nH. Market Risk\n1. Background\n2. Scope and Application of the Proposed\nRule\n3. Market Risk Covered Position\n4. Internal Risk Transfers\n5. General Requirements for Market Risk\n6. Measure for Market Risk\n7. Standardized Measure for Market Risk\n8. Models-Based Measure for Market Risk\n9. Treatment of Certain Market Risk\nCovered Positions\n10. Reporting and Disclosure Requirements\n11. Technical Amendments\nI. Credit Valuation Adjustment Risk\n1. Background\n2. Scope of Application\n3. CVA Risk Covered Positions and CVA\nHedges\n4. General Risk Management Requirements\n5. Measure for CVA Risk\nIV. Transition Provisions\nA. Transitions for Expanded Total RiskWeighted Assets\nB. AOCI Regulatory Capital Adjustments\nV. Impact and Economic Analysis\nA. Scope and Data\nB. Impact on Risk-Weighted Assets and\nCapital Requirements\nC. Economic Impact on Lending Activity\nD. Economic Impact on Trading Activity\nE. Additional Impact Considerations\nVI. Technical Amendments to the Capital\nRule\nA. Additional OCC Technical Amendments\nB. Additional FDIC Technical\nAmendments\nVII. Proposed Amendments to Related Rules\nand Related Proposals\nA. OCC Amendments\nB. Board Amendments\nC. Related Proposals\nVIII. Administrative Law Matters\nA. Paperwork Reduction Act\nB. Regulatory Flexibility Act\nC. Plain Language\nD. Riegle Community Development and\nRegulatory Improvement Act of 1994\nE. OCC Unfunded Mandates Reform Act of\n1995 Determination\nF. Providing Accountability Through\nTransparency Act of 2023"

In [ ]:
def get_summary_via_streaming_api_mr2(context: str) -> str:
    prompt = f"""For the context below, create a short summary with the most important bullet points only.\n\n{context}\n\nDo not repeat bullet points. Assign summaries into the following outline:\n{toc}. Make it crisp and concise."""
    headers = {
        "Authorization": f"Bearer {access_token[0]}",
        "Content-Type": "application/json; charset=utf-8"
    }
    
    data = {
        "inputs": [
            {
                "struct_val": {
                    "prompt": {
                        "string_val": [prompt]
                    }
                }
            }
        ],
        "parameters": {
            "struct_val": {
                "temperature": {"float_val": 0.0},
                "maxOutputTokens": {"int_val": 4096},
                "topK": {"int_val": 40},
                "topP": {"float_val": 0.8}
            }
        }
    }
    response = requests.post(STREAMING_API_URL_32K, headers=headers, json=data)
    content = json.loads(response.content)
    output = []

    for item in content:
        try:
            text = item['outputs'][0]['structVal']['content']['stringVal'][0]
            output.append(text)
        except Exception as e:
            logger.error(f'Content error => {content}')
    output = ''.join(output)
    return output

In [ ]:
CONTEXTS_PER_CALL = 50  # process 50 summaries per API call

def reduce_mr2(contexts: list) -> list:
    partitions = []
    max_input_tokens = CONTEXT_LENGTH - MAX_OUTPUT_TOKENS
    logger.info(f'Max input tokens allowed per API call = {max_input_tokens}')
    max_tokens_per_page = get_max_tokens_per_page(contexts)
    logger.info(f'Max tokens per page = {max_tokens_per_page}')
    logger.info(f'Processing {CONTEXTS_PER_CALL} pages per API call')
    
    for i in range(0, len(contexts), CONTEXTS_PER_CALL):
        partitions.append(contexts[i: i+CONTEXTS_PER_CALL])

    chunks = []
    for partition in partitions:
        chunks.append('\n'.join(partition))
    logger.info(f'Total number of chunks of summaries = {len(chunks)}')

    reduced_contexts = []

    with ThreadPoolExecutor(max_workers=1) as executor:  
        reduced_contexts = list(tqdm(executor.map(get_summary_via_streaming_api_mr2, chunks),  total=len(chunks)))
    return reduced_contexts

In [ ]:
reduced_summaries = reduce_mr2(summaries)

In [ ]:
logger.info(reduced_summaries[0])
logger.info('-' * 100)
logger.info(reduced_summaries[1])
logger.info('-' * 100)
logger.info(reduced_summaries[2])
logger.info('-' * 100)

##### Persist internediate summaries (Map Reduce 2) to local disk

In [ ]:
logger.info(f'Total number of summaries after map reduce 2 = {len(reduced_summaries)}')

In [ ]:
for i, summary in enumerate(reduced_summaries):
    os.makedirs(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/VAI/MAP_REDUCE_2/', exist_ok=True)
    with open(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/VAI/MAP_REDUCE_2/summary-{i}.txt', 'w') as f:
        f.write(summary)

In [ ]:
consolidated_summaries = '\n'.join(reduced_summaries)
logger.info(get_total_tokens([consolidated_summaries]))

In [ ]:
logger.info(consolidated_summaries)

#### Final Consolidation

In [ ]:
toc = "I. Introduction\nA. Overview of the Proposal\nB. Use of Internal Models Under the\nProposed Framework\nII. Scope of Application\nIII. Proposed Changes to the Capital Rule\nA. Calculation of Capital Ratios and\nApplication of Buffer Requirements\n1. Standardized Output Floor\n2. Stress Capital Buffer Requirement\nB. Definition of Capital\n1. Accumulated Other Comprehensive\nIncome\n2. Regulatory Capital Deductions\n3. Additional Definition of Capital\nAdjustments\n4. Changes to the Definition of Tier 2\nCapital Applicable to Large Banking\nOrganizations\nC. Credit Risk\n1. Due Diligence\n2. Proposed Risk Weights for Credit Risk\n3. Off-Balance Sheet Exposures\n4. Derivatives\n5. Credit Risk Mitigation\nD. Securitization Framework\n1. Operational Requirements\n2. Securitization Standardized Approach\n(SEC–SA)\n3. Exceptions to the SEC–SA Risk-Based\nCapital Treatment for Securitization\nExposures\n4. Credit Risk Mitigation for Securitization\nExposures\nE. Equity Exposures\n1. Risk-Weighted Asset Amount\nF. Operational Risk\n1. Business Indicator\n2. Business Indicator Component\n3. Internal Loss Multiplier\n4. Operational Risk Management and Data\nCollection Requirements\nG. Disclosure Requirements\n1. Proposed Disclosure Requirements\n2. Specific Public Disclosure Requirements\nH. Market Risk\n1. Background\n2. Scope and Application of the Proposed\nRule\n3. Market Risk Covered Position\n4. Internal Risk Transfers\n5. General Requirements for Market Risk\n6. Measure for Market Risk\n7. Standardized Measure for Market Risk\n8. Models-Based Measure for Market Risk\n9. Treatment of Certain Market Risk\nCovered Positions\n10. Reporting and Disclosure Requirements\n11. Technical Amendments\nI. Credit Valuation Adjustment Risk\n1. Background\n2. Scope of Application\n3. CVA Risk Covered Positions and CVA\nHedges\n4. General Risk Management Requirements\n5. Measure for CVA Risk\nIV. Transition Provisions\nA. Transitions for Expanded Total RiskWeighted Assets\nB. AOCI Regulatory Capital Adjustments\nV. Impact and Economic Analysis\nA. Scope and Data\nB. Impact on Risk-Weighted Assets and\nCapital Requirements\nC. Economic Impact on Lending Activity\nD. Economic Impact on Trading Activity\nE. Additional Impact Considerations\nVI. Technical Amendments to the Capital\nRule\nA. Additional OCC Technical Amendments\nB. Additional FDIC Technical\nAmendments\nVII. Proposed Amendments to Related Rules\nand Related Proposals\nA. OCC Amendments\nB. Board Amendments\nC. Related Proposals\nVIII. Administrative Law Matters\nA. Paperwork Reduction Act\nB. Regulatory Flexibility Act\nC. Plain Language\nD. Riegle Community Development and\nRegulatory Improvement Act of 1994\nE. OCC Unfunded Mandates Reform Act of\n1995 Determination\nF. Providing Accountability Through\nTransparency Act of 2023"

In [ ]:
def get_summary_via_streaming_api_mr3(context: str) -> str:
    prompt = f"""For the context below, group and consolidate content into abstracts within each applicable section.\n\n{context}\n\n."""
    #prompt = f"""Given the context below, combine and merge duplicate sections and pointers into the sections {toc}.\n\n{context}\nAdd SECTIONS and bullets wherever needed. Clean rewrite and re-number sections."""
    headers = {
        "Authorization": f"Bearer {access_token[0]}",
        "Content-Type": "application/json; charset=utf-8"
    }
    
    data = {
        "inputs": [
            {
                "struct_val": {
                    "prompt": {
                        "string_val": [prompt]
                    }
                }
            }
        ],
        "parameters": {
            "struct_val": {
                "temperature": {"float_val": 0.0},
                "maxOutputTokens": {"int_val": 8192},
                "topK": {"int_val": 40},
                "topP": {"float_val": 0.8}
            }
        }
    }
    response = requests.post(STREAMING_API_URL_32K, headers=headers, json=data)
    content = json.loads(response.content)
    output = []

    for item in content:
        try:
            text = item['outputs'][0]['structVal']['content']['stringVal'][0]
            output.append(text)
        except Exception as e:
            logger.error(f'Content error => {content}')
    output = ''.join(output)
    return output

In [ ]:
final_summary = get_summary_via_streaming_api_mr3(consolidated_summaries)
logger.info(final_summary)

In [ ]:
MAX_OUTPUT_TOKENS = 8192
max_input_tokens = CONTEXT_LENGTH - MAX_OUTPUT_TOKENS
logger.info(f'Max input tokens allowed per API call = {max_input_tokens}')
logger.info(f'Total tokens in final summary = {get_total_tokens([final_summary])}')

##### Persist final summary to local disk

In [ ]:
with open(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/VAI/final-summary.txt', 'w') as f:
    f.write(final_summary)

##### Create a filtered summary with all the proposed changes on the `Processing of Derivative Contracts`

In [ ]:
def get_summary_via_streaming_api_dc(context: str) -> str:
    prompt = f""""Given the context below, extract information only related processing of derivative contracts.\n\n{context}\n. Do not in include any other information."""
    headers = {
        "Authorization": f"Bearer {access_token[0]}",
        "Content-Type": "application/json; charset=utf-8"
    }
    
    data = {
        "inputs": [
            {
                "struct_val": {
                    "prompt": {
                        "string_val": [prompt]
                    }
                }
            }
        ],
        "parameters": {
            "struct_val": {
                "temperature": {"float_val": 0.0},
                "maxOutputTokens": {"int_val": 8192},
                "topK": {"int_val": 40},
                "topP": {"float_val": 0.8}
            }
        }
    }
    response = requests.post(STREAMING_API_URL_32K, headers=headers, json=data)
    content = json.loads(response.content)
    output = []

    for item in content:
        try:
            text = item['outputs'][0]['structVal']['content']['stringVal'][0]
            output.append(text)
        except Exception as e:
            logger.error(f'Content error => {content}')
    output = ''.join(output)
    return output

In [ ]:
proposed_changes_summary = get_summary_via_streaming_api_dc(consolidated_summaries)
logger.info(proposed_changes_summary)

In [ ]:
MAX_OUTPUT_TOKENS = 8192
max_input_tokens = CONTEXT_LENGTH - MAX_OUTPUT_TOKENS
logger.info(f'Max input tokens allowed per API call = {max_input_tokens}')
logger.info(f'Total tokens in final summary = {get_total_tokens([proposed_changes_summary])}')

In [ ]:
with open(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME}/VAI/proposed-changes-summary.txt', 'w') as f:
    f.write(proposed_changes_summary)